In [1]:
from Generate import Generate
from Traversal_Glove import Meta_Poetry_Glove

from nltk.corpus import wordnet as wn
from gensim.models import KeyedVectors

from functions import *
from model_back import Model as Model_back

import os
from six.moves import cPickle

In [2]:
poetic_vectors = KeyedVectors.load_word2vec_format('poetic_embeddings.300d.txt', binary=False)
#glove = KeyedVectors.load_word2vec_format("/Users/chris/Downloads/glove.6B/glove.6B.300d.w2v.txt")

In [3]:
g = Generate(wv=poetic_vectors, save_dir_back="combined_back")

In [4]:
mp = Meta_Poetry_Glove(wv=poetic_vectors)

# Experiments

In [5]:
import tensorflow as tf
from model_forw import Model as Model_forw
from model_back import Model as Model_back
import os
from six.moves import cPickle

In [6]:
def force_middle(model, vocab, prob_sequence, sequence, sess, lst, next_word):
    def decayRepeat(word,sequence, scale):
        safe_repeat_words = []
        #safe_repeat_words = set(["with,the,of,in,i"])
        score_adjust = 0
        decr = -scale
        for w in range(len(sequence)):
            if(word==sequence[w] and word not in safe_repeat_words):
                score_adjust += decr
            decr += scale/10 #decreases penalty as the words keep getting further from the new word
        return score_adjust

    poem = []
    line_num = 0
    wordPool_ind = 0
    ret=[]
    for tup in lst:
        state_init = tup[1][0][1]
        prob_seq =  tup[1][0][0]
        sequence=tup[1][1]
        scale = .02 #scale is the significant magnitude required to affect the score of bad/good things
        dist, state = model.compute_fx(sess, vocab, prob_seq, sequence, state_init, 1)
        #PREVENTS REPEAT ADJACENT WORDS OR PROBLEM-TAGGED WORDS
        if(next_word == sequence[-1]):
            continue
        #if(partsOfSpeechFilter(sequence[-1],next_word,dictPartSpeechTags,dictPossiblePartsSpeech)):
        #    continue
        #FACTORS IN SCORE ADJUSTMENTS
        score_adjust = decayRepeat(next_word, sequence, 100*scale) #repeats
        score_adjust += scale*len(next_word)/50 #length word
        #if(next_word in wordPool):
        #    score_adjust += scale
        #CALCULATES ACTUAL SCORE
        key = np.array([[vocab[next_word]]])
        new_prob = dist[key]
        score_tuple = (new_prob, state)
        score_tup = (score_tuple[0]+score_adjust,score_tuple[1]) #NOTE SHOULD SCORE_ADJUST BE ADDED HERE OR JUST IN THE ITEM LINE?
        item = (score_tup[0],(score_tup, sequence+[next_word]))
        if(item[0]==[[-float("inf")]]):
            continue
        ret+=[item]
    ret.sort(key=itemgetter(0), reverse = True)
    return ret[:g.get_width()]


In [7]:
def generate_line_collocations(word1, word2, collocations):
    # check for collocations and fill in
    postag= g.pos_synset([word1, word2], g.get_postag_dict()[0])

    last_position=np.argmax(postag[1])
    first_position=np.argmin(postag[1])

    last_noun=postag[2][last_position]
    first_noun=postag[2][first_position]

    first_pos=postag[1][first_position]
    last_pos=postag[1][last_position]
    template=postag[0]

    line = ['' for i in range(len(template))]
    line[min(postag[1])] = first_noun
    line[max(postag[1])] = last_noun
    
    line = g.insert_collocations(template, line, collocations)
    
    # finding indices with words in them
    not_blank = []
    pairs = []
    for i in range(len(line)):
        if line[i] is not '':
            pairs.append(i)
        if len(pairs) > 1:
            not_blank.append((pairs[0], pairs[1]))
            pairs.pop(0)
    # if haven't made it to the end of the sentence
    if pairs[0] != len(line) - 1:
        not_blank.append((pairs[0], len(line) - 1))

    # iterating over pairs of filled in words, filling in between
    candidates = []

    first_time = True
    first_noun = line[not_blank[0][0]]
    first_pos = not_blank[0][0]
    for i, j in not_blank:
        last_noun = line[j]
        last_pos = j
        
        ### Matias's code
        
        if first_time:
            list_1= g.genPoem_forward(first_noun, template[first_pos:last_pos])
            list_2= g.force_middle(list_1[:50], last_noun)
            candidates.extend(list_2)
            first_time = False
            continue

        tf.reset_default_graph()
        with open(os.path.join(g.get_save_dir(), 'config.pkl'), 'rb') as f:
            saved_args = cPickle.load(f)
        with open(os.path.join(g.get_save_dir(), 'words_vocab.pkl'), 'rb') as f:
            word_keys, vocab = cPickle.load(f)
        model = Model_forw(saved_args, True)
        with tf.Session() as sess:
            tf.global_variables_initializer().run()
            saver = tf.train.Saver(tf.global_variables())
            ckpt = tf.train.get_checkpoint_state(g.get_save_dir())
            if ckpt and ckpt.model_checkpoint_path:
                saver.restore(sess, ckpt.model_checkpoint_path)
                poem = []
                line_num = 0
                wordPool_ind = 0
                tt_3=[]
                for element in candidates:
                    seq=element[1][1]
                    state = element[1][0][1]
                    init_score = element[1][0][0]
                    seq=element[1][1]
                    lst = search_forward(model, vocab, init_score,seq,state, sess, 1,\
                                      g.get_dict_tag(),g.get_dict_pos(),g.get_width(),
                                         g.get_wordPools()[wordPool_ind], g.get_pos_set(), template[first_pos:last_pos])
                    # case where are generating between
                    # two words and need to force last one
                    if last_noun is not '':
                        lst = force_middle(model, vocab, init_score, seq, sess, lst[:50], last_noun)
                    tt_3+=lst
                tt_3.sort(key=itemgetter(0), reverse = True)
                candidates = tt_3[:50]
    pre_backwards = candidates
        
    # Filling in reverse
    tf.reset_default_graph()
    with open(os.path.join(g.get_save_dir_back(), 'config.pkl'), 'rb') as f:
        saved_args = cPickle.load(f)
    with open(os.path.join(g.get_save_dir_back(), 'words_vocab.pkl'), 'rb') as f:
        word_keys, vocab = cPickle.load(f)
    model = Model_back(saved_args, True)
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        saver = tf.train.Saver(tf.global_variables())
        ckpt = tf.train.get_checkpoint_state(g.get_save_dir_back())
        if ckpt and ckpt.model_checkpoint_path:
            saver.restore(sess, ckpt.model_checkpoint_path)
            poem = []
            line_num = 0
            wordPool_ind = 0
            tt_4=[]
            for tup in candidates[:30]:
                seq=tup[1][1]
                init_score, state=model.score_a_list(sess, vocab, seq)
                lst = search_back(model, vocab, init_score,seq,state, sess, 1,\
                              g.get_dict_tag(), g.get_dict_pos(),g.get_width(),
                                  g.get_wordPools()[wordPool_ind], g.get_pos_set(), template)

                tt_4+=lst
        else:
            print(ckpt)
    tt_4.sort(key=itemgetter(0), reverse = True)
    candidates = tt_4

    return postag, not_blank, line, candidates, pre_backwards

In [18]:
# %%time

# poets = {'Frost':'Grammar/Robert_Frost.txt'}
# filters = [("NN", "NN"), ("JJ", "NN"), ("JJ", "JJ", "NN"), ("JJ", "NN", "NN"), ("NN", "JJ", "NN"), ("NN", "NN", "NN"), ("NN", "PRP", "NN")]
# colls_dict = build_get_collocation_dict(poets, filters=filters)

# with open('colls_dict.p', 'wb') as f:
#     cPickle.dump(colls_dict, f)

In [12]:
with open('colls_dict.p', 'rb') as f:
    colls_dict = cPickle.load(f)

In [13]:
colls_dict['clock']

[[('clock', 'i'), -4.0, 1.0],
 [('clock', 'think'), -4.0, 0.0],
 [('clock', 'hear'), -2.0, 0.0],
 [('clock', 'rattling.'), 3.0, 0.0],
 [('clock', 'maybe/'), 1.0, 0.0],
 [('clock', "shelf's"), -3.0, 0.0],
 [('clock', 'vase'), 2.0, 0.0],
 [('clock', 'picture,'), 4.0, 0.0],
 [('clock', 'luminary'), -1.0, 0.0],
 [('clock', 'sky'), 3.0, 0.0]]

In [14]:
colls_dict['time']

[[('time', 'full'), -3.0, 0.0],
 [('time', 'many'), -2.0, 0.0],
 [('time', 'doubt'), 3.0, 0.0],
 [('time', 'river'), -3.0, 0.0],
 [('time', 'i'), 1.0, 0.0],
 [('time', 'fellow'), 4.0, 0.0],
 [('time', 'dollar'), -3.0, 0.0],
 [('time', 'grow'), 5.0, 0.0],
 [('time', 'saw'), 3.0, 0.0],
 [('time', "'it's"), -1.0, 0.0],
 [('time', 'turn'), 4.0, 0.0],
 [('time', "i'm"), 3.0, 0.0],
 [('time', 'sort'), -4.0, 0.0],
 [('time', 'swear'), -2.0, 0.0],
 [('time', 'awful!'), 3.0, 0.0],
 [('time', 'think'), 2.0, 0.0],
 [('time', 'something'), 4.0, 0.0],
 [('time', 'child'), -4.0, 0.0],
 [('time', "night?'"), 2.0, 0.0],
 [('time', 'growth,'), 3.0, 0.0],
 [('time', 'talk'), 2.0, 0.0],
 [('time', 'talk.'), 2.0, 0.0],
 [('time', 'cow'), -3.0, 0.0],
 [('time', 'apple'), -1.0, 0.0],
 [('time', 'flight:'), 5.0, 0.0],
 [('time', 'spread'), -5.0, 0.0],
 [('time', 'coal-black'), -4.0, 0.0],
 [('time', 'particular'), -1.0, 0.0],
 [('time', "o'clock,"), 1.0, 0.0],
 [('time', 'thousandth'), -1.0, 0.0],
 [('time',

In [15]:
line = generate_line_collocations('time', 'clock', colls_dict)

['time', 'clock']
['NN', 'NN']
Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').
6885
INFO:tensorflow:Restoring parameters from save_2/model.ckpt-63199
[(array([[0]]), ((array([[0]]), (LSTMStateTuple(c=array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 

6885
INFO:tensorflow:Restoring parameters from save_2/model.ckpt-63199
INFO:tensorflow:Restoring parameters from save_2/model.ckpt-63199
INFO:tensorflow:Restoring parameters from frost_model/model.ckpt-227699


/Users/chris/projects/poetix/model_back.py:207: RuntimeWarning: divide by zero encountered in log
  dist = p.squeeze() + np.log(probs.squeeze())


In [16]:
line[1]

[(4, 5), (5, 6)]

In [17]:
line[2]

['', '', '', '', 'clock', 'time', '']

In [18]:
line[0]

[['VBG', 'IN', 'IN', 'DT', 'NN', 'NN', ','], [5, 4], ['time', 'clock']]

In [19]:
for i in range(len(line[3])):
    print(' {:0.3}: '.format(line[3][i][0][0][0]), end='')
    print(' '.join(line[3][i][1][1]))

 -46.7: bough on the snow mowing clock time

 -49.5: afraid in the apple wing clock time

 -49.7: afraid in an apple wing clock time

 -50.9: bough in the snow mowing clock time

 -51.2: afraid on the apple wing clock time

 -51.5: afraid in the kitchen wing clock time

 -51.5: bough against the snow mowing clock time

 -51.7: afraid in the thick wing clock time

 -51.8: afraid in this apple wing clock time

 -51.9: afraid against the snow mowing clock time

 -52.3: afraid in the light mowing clock time

 -53.0: afraid in the snow mowing clock time

 -53.2: bough in the kitchen wing clock time

 -53.3: afraid about the snow mowing clock time

 -53.6: past with the thick wing clock time

 -53.6: afraid on the thick wing clock time

 -53.8: afraid in a apple wing clock time

 -53.9: past in the kitchen wing clock time

 -53.9: bough in the thick wing clock time

 -53.9: afraid on the snow mowing clock time



# Using nltk's collocations

In [6]:
from nltk.collocations import *
from nltk.corpus import stopwords
import string
from nltk.metrics import BigramAssocMeasures
import codecs

In [67]:
poets = ['endymion', 'frost', 'modern_poetry', 'whitman', 'wilde', 'ya', 'ya_all']

stop = set(stopwords.words('english'))
punct = str.maketrans('', '', string.punctuation)
tokens=[]

for poet in poets:
    print(poet)
    f = codecs.open(os.path.join('data', poet, 'input.txt'), 'r', encoding='utf-8', errors='replace')
    raw = f.read()
    f.close
    raw = raw.translate(punct).lower()
    tokens.extend(nltk.word_tokenize(raw))

endymion
frost
modern_poetry
whitman
wilde
ya
ya_all


In [68]:
finder = BigramCollocationFinder.from_words(tokens, window_size=3)

In [69]:
finder.nbest(BigramAssocMeasures.likelihood_ratio, 10)

[('the', 'the'),
 ('the', 'of'),
 ('had', 'been'),
 ('i', 'am'),
 ('more', 'than'),
 ('in', 'the'),
 ('a', 'the'),
 ('of', 'the'),
 ('the', 'a'),
 ('i', 'say')]

In [73]:
colls = finder.score_ngrams(BigramAssocMeasures.likelihood_ratio)
filters = [("NN", "NN"), ("JJ", "NN")]

coll_dict = {}
for coll, score in colls:
    if score < 5:
        continue
    
    if len(g.get_postag_dict()[2][coll[0]]) == 0 or len(g.get_postag_dict()[2][coll[1]]) == 0:
        continue

    pos = (g.get_postag_dict()[2][coll[0]][0], g.get_postag_dict()[2][coll[1]][0])
    
    # POS heuristic filter
    if pos not in filters:
        continue
    
    if coll[0] not in coll_dict:
        coll_dict[coll[0]] = []

    coll_dict[coll[0]].append((coll[1], score))

for value in coll_dict.values():
    value.sort(key=itemgetter(1), reverse=True)
coll_dict

{'first': [('time', 333.92606018149223),
  ('day', 37.291406792120874),
  ('impulse', 36.604990307949755),
  ('thing', 36.063309691993666),
  ('step', 21.129448496108246),
  ('meeting', 17.163701667604347),
  ('year', 13.52824435287113),
  ('glance', 13.038910164047515),
  ('line', 12.459724712840453),
  ('gloveless', 11.435044717932533),
  ('haycock', 11.435044717932533),
  ('letter', 10.418750631869091),
  ('ray', 10.255005513416657),
  ('gift', 9.886347939132023),
  ('motor', 9.712242589362534),
  ('propensity', 9.712242589362534),
  ('kite', 9.20854466635326),
  ('outcry', 9.20854466635326),
  ('visit', 7.37708882736959),
  ('slab', 7.174806491416104),
  ('car', 7.009792683358835),
  ('night', 6.752969445189985),
  ('breath', 6.7439160560507245),
  ('intent', 6.6258962796485275),
  ('place', 6.572937381975494),
  ('page', 6.537417351740823),
  ('suggestion', 6.104611177315627),
  ('stung', 5.929977175833011),
  ('street', 5.217710914826366),
  ('choice', 5.173847150354977)],
 'youn

In [75]:
# Assumes sorted by score
def insert_collocations(template, line, collocations, window_width=3):
    for i, w in enumerate(line):
        if w not in collocations:
            continue;

        grams = collocations[w]
        # iterating over all collocations
        for gram in grams:
            col_word = gram[0]

            if len(g.get_postag_dict()[2][col_word]) == 0:
                continue

            gram_pos = g.get_postag_dict()[2][col_word]

            # empty spot and pos matches
            for j in range(i+1, len(line)):
                if line[j] is '' and template[j] in gram_pos and j - i <= window_width:
                    line[j] = col_word
                    break
            # Break out of second loop if broke from third loop
            else:
                continue
            break
    return line

insert_collocations(['PRP', 'VBP', 'TO', 'VB', 'DT', 'JJ', 'NN', 'IN', 'DT', 'NN'], 
                    ['I', '', '', '', '', 'open', '', '', '', ''], coll_dict)

['I', '', '', '', '', 'open', 'air', '', '', 'water']

In [76]:
def generate_line_nltk_collocations(word1, word2, collocations):
    # check for collocations and fill in
    postag= g.pos_synset([word1, word2], g.get_postag_dict()[0])

    last_position=np.argmax(postag[1])
    first_position=np.argmin(postag[1])

    last_noun=postag[2][last_position]
    first_noun=postag[2][first_position]

    first_pos=postag[1][first_position]
    last_pos=postag[1][last_position]
    template=postag[0]

    line = ['' for i in range(len(template))]
    line[min(postag[1])] = first_noun
    line[max(postag[1])] = last_noun
    
    line = insert_collocations(template, line, collocations)
    
    # finding indices with words in them
    not_blank = []
    pairs = []
    for i in range(len(line)):
        if line[i] is not '':
            pairs.append(i)
        if len(pairs) > 1:
            not_blank.append((pairs[0], pairs[1]))
            pairs.pop(0)
    # if haven't made it to the end of the sentence
    if pairs[0] != len(line) - 1:
        not_blank.append((pairs[0], len(line) - 1))

    # iterating over pairs of filled in words, filling in between
    candidates = []

    first_time = True
    first_noun = line[not_blank[0][0]]
    first_pos = not_blank[0][0]
    for i, j in not_blank:
        last_noun = line[j]
        last_pos = j
        
        ### Matias's code
        
        if first_time:
            list_1= g.genPoem_forward(first_noun, template[first_pos:last_pos])
            list_2= g.force_middle(list_1[:50], last_noun)
            candidates.extend(list_2)
            first_time = False
            continue

        tf.reset_default_graph()
        with open(os.path.join(g.get_save_dir(), 'config.pkl'), 'rb') as f:
            saved_args = cPickle.load(f)
        with open(os.path.join(g.get_save_dir(), 'words_vocab.pkl'), 'rb') as f:
            word_keys, vocab = cPickle.load(f)
        model = Model_forw(saved_args, True)
        with tf.Session() as sess:
            tf.global_variables_initializer().run()
            saver = tf.train.Saver(tf.global_variables())
            ckpt = tf.train.get_checkpoint_state(g.get_save_dir())
            if ckpt and ckpt.model_checkpoint_path:
                saver.restore(sess, ckpt.model_checkpoint_path)
                poem = []
                line_num = 0
                wordPool_ind = 0
                tt_3=[]
                for element in candidates:
                    seq=element[1][1]
                    state = element[1][0][1]
                    init_score = element[1][0][0]
                    seq=element[1][1]
                    lst = search_forward(model, vocab, init_score,seq,state, sess, 1,\
                                      g.get_dict_tag(),g.get_dict_pos(),g.get_width(),
                                         g.get_wordPools()[wordPool_ind], g.get_pos_set(), template[first_pos:last_pos])
                    # case where are generating between
                    # two words and need to force last one
                    if last_noun is not '':
                        lst = force_middle(model, vocab, init_score, seq, sess, lst[:50], last_noun)
                    tt_3+=lst
                tt_3.sort(key=itemgetter(0), reverse = True)
                candidates = tt_3[:50]
    pre_backwards = candidates
        
    # Filling in reverse
    tf.reset_default_graph()
    with open(os.path.join(g.get_save_dir_back(), 'config.pkl'), 'rb') as f:
        saved_args = cPickle.load(f)
    with open(os.path.join(g.get_save_dir_back(), 'words_vocab.pkl'), 'rb') as f:
        word_keys, vocab = cPickle.load(f)
    model = Model_back(saved_args, True)
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        saver = tf.train.Saver(tf.global_variables())
        ckpt = tf.train.get_checkpoint_state(g.get_save_dir_back())
        if ckpt and ckpt.model_checkpoint_path:
            saver.restore(sess, ckpt.model_checkpoint_path)
            poem = []
            line_num = 0
            wordPool_ind = 0
            tt_4=[]
            for tup in candidates[:30]:
                seq=tup[1][1]
                init_score, state=model.score_a_list(sess, vocab, seq)
                lst = search_back(model, vocab, init_score,seq,state, sess, 1,\
                              g.get_dict_tag(), g.get_dict_pos(),g.get_width(),
                                  g.get_wordPools()[wordPool_ind], g.get_pos_set(), template)

                tt_4+=lst
        else:
            print(ckpt)
    tt_4.sort(key=itemgetter(0), reverse = True)
    candidates = tt_4

    return postag, not_blank, line, candidates, pre_backwards

In [77]:
line = generate_line_nltk_collocations('time', 'clock', coll_dict)

['time', 'clock']
['NN', 'NN']
6885
INFO:tensorflow:Restoring parameters from save_2/model.ckpt-63199
[(array([[0]]), ((array([[0]]), (LSTMStateTuple(c=array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

6885
INFO:tensorflow:Restoring parameters from save_2/model.ckpt-63199
INFO:tensorflow:Restoring parameters from frost_model/model.ckpt-227699


/Users/chris/projects/poetix/model_back.py:207: RuntimeWarning: divide by zero encountered in log
  dist = p.squeeze() + np.log(probs.squeeze())


In [78]:
for i in range(len(line[3])):
    print(' {:0.3}: '.format(line[3][i][0][0][0]), end='')
    print(' '.join(line[3][i][1][1]))

 -69.4: living eyes have ago while clock time
 -70.3: something eyes have ago while clock time
 -70.3: something years have ago wing clock time
 -70.7: moving years have ago wing clock time
 -70.7: living leaves have ago while clock time
 -70.8: living wheels have ago wing clock time
 -71.4: misting leaves have ago while clock time
 -71.4: living years have ago wing clock time
 -71.5: being eyes have ago while clock time
 -71.7: breathing trees have ago while clock time
 -71.8: something wheels have ago while clock time
 -71.9: moving leaves have ago while clock time
 -72.3: moving trees have ago while clock time
 -72.4: passing eyes have ago while clock time
 -72.4: blowing trees have ago while clock time
 -72.5: saying eyes have ago while clock time
 -72.5: saving years have ago wing clock time
 -72.7: keeping wheels have ago wing clock time
 -72.8: gathering trees have awhile while clock time
 -72.8: coming years have ago wing clock time


# Rhyming WordNet Traversal

In [5]:
import tensorflow as tf
from model_forw import Model as Model_forw
from model_back import Model as Model_back
import os
from six.moves import cPickle
import requests
from functions import *

In [6]:
# word1 -> (rhyme)
#    \     /
#     word3

mp.traverse_wn_glove('gallery')

('art', 'a room or series of rooms where works of art are exhibited')

In [7]:
params ={'rel_rhy': 'salary'}
response = requests.get('https://api.datamuse.com/words', params=params)

In [8]:
response.json()

[{'word': 'gallery', 'score': 1571, 'numSyllables': 3},
 {'word': 'calorie', 'score': 617, 'numSyllables': 3},
 {'word': 'peanut gallery', 'score': 292, 'numSyllables': 5},
 {'word': 'valerie', 'score': 283, 'numSyllables': 3},
 {'word': 'mallory', 'score': 156, 'numSyllables': 3},
 {'word': 'callery', 'score': 127, 'numSyllables': 3},
 {'word': 'valery', 'score': 85, 'numSyllables': 3},
 {'word': 'art gallery', 'score': 81, 'numSyllables': 4},
 {'word': 'shooting gallery', 'score': 42, 'numSyllables': 5},
 {'word': 'mallery', 'score': 34, 'numSyllables': 3},
 {'word': 'whispering gallery', 'score': 29, 'numSyllables': 6},
 {'word': 'gram calorie', 'score': 21, 'numSyllables': 4},
 {'word': 'picture gallery', 'score': 20, 'numSyllables': 5},
 {'word': 'press gallery', 'score': 20, 'numSyllables': 4},
 {'word': 'small calorie', 'score': 20, 'numSyllables': 4},
 {'word': 'kilogram calorie', 'score': 19, 'numSyllables': 6},
 {'word': 'large calorie', 'score': 17, 'numSyllables': 4},
 {'wo

In [9]:
from gensim.parsing.preprocessing import remove_stopwords
from nltk.corpus import wordnet as wn

def two_word_link(w1, w2):
    max_sim = -1
    best_word = None
    best_word_def = None
    
    word_set = set()
    
    for synset in wn.synsets(w1):
        clean_def = remove_stopwords(mp.punct.sub('', synset.definition()))
        word_set.update(clean_def.lower().split())
    for synset in wn.synsets(w2):
        clean_def = remove_stopwords(mp.punct.sub('', synset.definition()))
        word_set.update(clean_def.lower().split())
        
    for other_word in word_set:
        sim = mp.get_glove_sim(w1, other_word)
        sim += mp.get_glove_sim(w2, other_word)
        
        if sim > max_sim and other_word != w1 and other_word != w2:
            max_sim = sim
            best_word = other_word
            
    return best_word

In [10]:
two_word_link('salary', 'gallery')

/Users/chris/projects/poetix/Traversal_Glove.py:44: RuntimeWarning: invalid value encountered in true_divide
  avg_w2 /= np.sqrt(np.sum(avg_w2 ** 2))


'art'

In [12]:
def get_five_words(w2, pos_to_words):
    # three connection words
    w_response = requests.get('https://api.datamuse.com/words', params={'rel_rhy': w2}).json()
    rhyme_nnp = set(d['word'] for d in w_response).intersection(pos_to_words['NNP'])
    
    if len(w_response) < 2 or len(rhyme_nnp) < 1:
        raise ValueError('Cannot generate limkerick using ', w2)
        
    w5 = w_response[0]['word']
    w4 = two_word_link(w2, w5)
    
    w1 = list(rhyme_nnp)[0]

    w3_response = requests.get('https://api.datamuse.com/words', params={'rel_rhy': w4}).json()
    
    if len(w3_response) < 1:
        raise ValueError('Cannot generate limkerick using ', w2)
        
    w3 = w3_response[0]['word']
    return w1, w2, w3, w4, w5

In [13]:
# print(get_five_words('salary', g.get_postag_dict()[1]))
# print(get_five_words('chicken', g.get_postag_dict()[1]))
# print(get_five_words('battery', g.get_postag_dict()[1]))
print(get_five_words('phone', g.get_postag_dict()[1]))
print(get_five_words('tree', g.get_postag_dict()[1]))
print(get_five_words('war', g.get_postag_dict()[1]))
print(get_five_words('car', g.get_postag_dict()[1]))

('joan', 'phone', 'bone', 'telephone', 'bone')
('t', 'tree', 'disease', 'trees', 'be')
('centaur', 'war', 'bend', 'end', 'for')


ValueError: ('Cannot generate limkerick using ', 'car')

In [14]:
def gen_line(w1, template=None, pronoun=None):
    if template is None:
        w1_pos = g.get_postag_dict()[2][w1][0]
        template = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k']
        while len(template) > 10:
            template = np.random.choice(templates_dict[w1_pos], 1).item()
    
    tf.reset_default_graph()
    with open(os.path.join(g.get_save_dir_back(), 'config.pkl'), 'rb') as f:
        saved_args = cPickle.load(f)
    with open(os.path.join(g.get_save_dir_back(), 'words_vocab.pkl'), 'rb') as f:
        word_keys, vocab = cPickle.load(f)
    model = Model_back(saved_args, True)
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        saver = tf.train.Saver(tf.global_variables())
        ckpt = tf.train.get_checkpoint_state(g.get_save_dir_back())
        if ckpt and ckpt.model_checkpoint_path:
            saver.restore(sess, ckpt.model_checkpoint_path)
            poem = []
            line_num = 0
            wordPool_ind = 0
            state = sess.run(model.initial_state)
            init_score = np.array([[0]])

            lst = search_back(model, vocab, init_score,[w1],state, sess, 1,
                          g.get_dict_tag(), g.get_dict_pos(),g.get_width(),
                              g.get_wordPools()[wordPool_ind], g.get_pos_set(), template)
            lst.sort(key=itemgetter(0), reverse = True)
        else:
            raise IOError('No checkpoint')
    return template, lst

In [15]:
templates_dict = {}

for l in g.get_postag_dict()[0].values():
    for t, _ in l:
        ending_pos = t[-1]
        if ending_pos not in templates_dict:
            templates_dict[ending_pos] = []
        templates_dict[ending_pos].append(t)

In [16]:
def template_generalization(templates_dict, words, n_words=10):
    scored_dict = {}
    for pos, templates in templates_dict.items():
        scored_dict[pos] = []
        # random words with pos w/o replacement
        if len(words[pos]) < n_words:
            continue
        words = np.random.choice(words[pos], n_words)
        for t in templates:
            template_score = 0
            for w in words:
                print(w, t)
                _, out = gen_line(w, template = t)
                template_score += out[0][0].item() / len(t)
            scored_dict[pos].append((template_score / n_words, t))
    return scored_dict

In [17]:
def gen_poem(w):
    five_words = get_five_words(w, g.get_postag_dict()[1])
    
    first_line = ['There', 'once', 'was', 'a', 'kind', 'old']
    first_template = ['EX', 'RB', 'VBD', 'DT', 'JJ', 'JJ', 'NNP']
    first_line.append(five_words[0])
    
    lines = []
    lines.append((first_line, 0, first_template))
    for i, w in enumerate(five_words):
        if i == 0:
            continue
        t, out = gen_line(w, pronoun='he')
    
        this_line = out[0][1][1]
        this_score = out[0][0].item()
        lines.append((this_line, this_score, t))
    return lines

In [18]:
get_five_words('war', g.get_postag_dict()[1])

('centaur', 'war', 'bend', 'end', 'for')

In [19]:
nltk.pos_tag(['There', 'once', 'was', 'a', 'kind', 'old', 'Labrador'])


[('There', 'EX'),
 ('once', 'RB'),
 ('was', 'VBD'),
 ('a', 'DT'),
 ('kind', 'NN'),
 ('old', 'JJ'),
 ('Labrador', 'NN')]

In [34]:
%%time

lines = gen_poem('war')

/Users/chris/projects/poetix/Traversal_Glove.py:44: RuntimeWarning: invalid value encountered in true_divide
  avg_w2 /= np.sqrt(np.sum(avg_w2 ** 2))


INFO:tensorflow:Restoring parameters from combined_back/model.ckpt-217959
INFO:tensorflow:Restoring parameters from combined_back/model.ckpt-217959
INFO:tensorflow:Restoring parameters from combined_back/model.ckpt-217959
INFO:tensorflow:Restoring parameters from combined_back/model.ckpt-217959
CPU times: user 52.4 s, sys: 17.7 s, total: 1min 10s
Wall time: 1min 6s


In [35]:
for l, s, t in lines:
    print('{:50} line score: {:2.3f}'.format(' '.join(l), s))
    print(t)

There once was a kind old centaur                  line score: 0.000
['EX', 'RB', 'VBD', 'DT', 'JJ', 'JJ', 'NNP']
come eos here at times day and evening war         line score: -42.267
['NNP', 'NN', 'RB', 'IN', 'NNS', 'NN', 'CC', 'NN', 'NN']
few days of things , you bend                      line score: -25.955
['JJ', 'NNS', 'IN', 'NNS', ',', 'PRP', 'VBP']
in eos long evening north and end                  line score: -32.360
['IN', 'NNP', 'JJ', 'NN', 'NN', 'CC', 'NN']
hands of my mind come home for                     line score: -26.295
['NNS', 'IN', 'PRP$', 'NN', 'NNP', 'NN', 'IN']
